Libraries

In [2]:
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup
import requests
import pandas as pd
from sqlalchemy import create_engine
import time
import selenium
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver import ActionChains
from selenium.common.exceptions import TimeoutException


Scraper

In [6]:
#Setting up Webdriver and options
ChromeOptions = webdriver.ChromeOptions()
ChromeOptions.add_argument('--disable-browser-side-navigation')
ChromeOptions.add_argument('--headless') # This makes Webdriver NOT open a browser, it is needed to reduce load on PC
driver = webdriver.Chrome(options=ChromeOptions)

#Getting a link
driver.get("https://www.rightmove.co.uk/property-for-sale/find.html?searchType=SALE&locationIdentifier=REGION%5E162&insId=1&radius=0.0&minPrice=&maxPrice=&minBedrooms=&maxBedrooms=&displayPropertyType=&maxDaysSinceAdded=&_includeSSTC=on&sortByPriceDescending=&primaryDisplayPropertyType=&secondaryDisplayPropertyType=&oldDisplayPropertyType=&oldPrimaryDisplayPropertyType=&newHome=&auction=false")
print(driver.title)

driver.maximize_window()

dicts= {}
df = pd.DataFrame()

#We wait for the cookies 
wait = WebDriverWait(driver, 10)
wait.until(EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/div[2]/div[4]/div[2]/div/button")))

print(f"-----Main menu:{driver.current_url}---")

#Clicking "Agree" button on cookies
time.sleep(1)
driver.find_element(By.XPATH, "/html/body/div[1]/div[2]/div[4]/div[2]/div/button").click()
print("cookies gone")

#collecting list of Ads on a page should be 25 per page
elements = driver.find_elements(By.CSS_SELECTOR, "a[class^='propertyCard-link property-card-updates']")
print("-----------")

#first loop is for clicking through pages
for i in range(30):
    #second loop is for cicking through Ads on a page 
    for j in range(len(elements)):
        
        dicts={}
        
        #We collect elements again as otherwise data goes stale
        #So elements list will get refreshed everytime we go back and forth between ads
        wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "a[class^='propertyCard-link property-card-updates']")))
        elements = driver.find_elements(By.CSS_SELECTOR, "a[class^='propertyCard-link property-card-updates']")
        
        url = []
        print(len(elements))
        #We collect the link for the Ad
        url = elements[j].get_attribute('href')
        print(f'url{j}:{url}')
        
        #We click on next Ad through the link
        driver.get(url)
        
        #Now we start collecting data
        driver.implicitly_wait(2)
        values= driver.find_elements(By.CLASS_NAME, "_1hV1kqpVceE9m-QrX_hWDN  ")
        print("fetched values")
        keys= driver.find_elements(By.CLASS_NAME, "ZBWaPR-rIda6ikyKpB_E2")
        print("fetched keys")
        add = driver.find_element(By.CSS_SELECTOR, 'h1._2uQQ3SV0eMHL1P6t5ZDo2q')
        print(add.text)
        pr = driver.find_element(By.CLASS_NAME, '_1gfnqJ3Vtd1z40MlC0MzXu')
        print(pr.text)
        
        #Coordinates
        coord1 = driver.find_element(By.CSS_SELECTOR, 'a[class="_1kck3jRw2PGQSOEy3Lihgp"]')
        coord2 = coord1.find_element(By.TAG_NAME,'img').get_attribute('src')
        print(coord2)
        #We find coordinates thorugh slicing the URL of an image, not a perfect way.
        lat = coord2[87:96]
        lon = coord2[104:120]
        print(lat)
        print(lon)
        for p in range(len(keys)):
            #appending dictionary with values
            dicts[keys[p].text] = values[p].text
            
            dicts['Address'] = add.text 
            dicts['Price'] = pr.text
            dicts['Latitude'] = lat
            dicts['Longitude'] = lon
        #appending dataframe with dictionary values
        df = df.append(dicts, ignore_index=True)
        
        driver.implicitly_wait(2)
        print(f"---Currently on:{driver.current_url}----")
        #Now we go back
        driver.back()
        print(f"---===Went back, Currently on:{driver.current_url}===----")

        
    #Next Page button    
    driver.find_element(By.CSS_SELECTOR, "button[class='pagination-button pagination-direction pagination-direction--next']").click()
    print(f"-------moving to next page, Page number: {i} url:{driver.current_url}--------")

Properties For Sale in Birmingham | Rightmove
-----Main menu:https://www.rightmove.co.uk/property-for-sale/find.html?searchType=SALE&locationIdentifier=REGION%5E162&insId=1&radius=0.0&minPrice=&maxPrice=&minBedrooms=&maxBedrooms=&displayPropertyType=&maxDaysSinceAdded=&_includeSSTC=on&sortByPriceDescending=&primaryDisplayPropertyType=&secondaryDisplayPropertyType=&oldDisplayPropertyType=&oldPrimaryDisplayPropertyType=&newHome=&auction=false---
cookies gone
-----------
25
https://www.rightmove.co.uk/properties/119545151#/?channel=RES_BUY
url0:https://www.rightmove.co.uk/properties/119545151#/?channel=RES_BUY
fetched values
fetched keys
Sydenham Place, 26B Tenby Street , Jewellery Quarter
£449,950
<selenium.webdriver.remote.webelement.WebElement (session="1a61949b69cf07612d04cada3bb45c47", element="c4089b1e-9030-4d8a-ac9a-04c86a77c255")>

https://media.rightmove.co.uk/map/_generate?width=768&height=347&zoomLevel=15&latitude=52.48575&longitude=-1.91351&signature=KTg-u-W_SHPFAP8gsMFwH4tdEP

fetched values
fetched keys
Wellington Road, Edgbaston
£2,250,000
<selenium.webdriver.remote.webelement.WebElement (session="1a61949b69cf07612d04cada3bb45c47", element="b9513909-caea-420a-9fae-01a96127e4f2")>

https://media.rightmove.co.uk/map/_generate?width=768&height=347&zoomLevel=15&latitude=52.462935&longitude=-1.909706&signature=y0kQImj_2Pi8S3sUrQ5N4-LKODI=
https://media.rightmove.co.uk/map/_generate?width=768&height=347&zoomLevel=15&latitude=52.462935&longitude=-1.909706&signature=y0kQImj_2Pi8S3sUrQ5N4-LKODI=
52.462935
de=-1.909706&sig
{'PROPERTY TYPE': 'Detached'}
{'PROPERTY TYPE': 'Detached', 'Address': 'Wellington Road, Edgbaston', 'Price': '£2,250,000', 'Latitude': '52.462935', 'Longitude': 'de=-1.909706&sig', 'BEDROOMS': '×6'}
{'PROPERTY TYPE': 'Detached', 'Address': 'Wellington Road, Edgbaston', 'Price': '£2,250,000', 'Latitude': '52.462935', 'Longitude': 'de=-1.909706&sig', 'BEDROOMS': '×6', 'BATHROOMS': '×3'}
{'PROPERTY TYPE': 'Detached', 'Address': 'Wellington Road, Edg

fetched values
fetched keys
Harborne Road, Edgbaston
£1,699,950
<selenium.webdriver.remote.webelement.WebElement (session="1a61949b69cf07612d04cada3bb45c47", element="8b93d1a2-fe09-4662-9145-2bca7b753561")>

https://media.rightmove.co.uk/map/_generate?width=768&height=347&zoomLevel=15&latitude=52.469913&longitude=-1.92736&signature=4lj2qj1xUhBRFJmzlY0iLfJc3cA=
https://media.rightmove.co.uk/map/_generate?width=768&height=347&zoomLevel=15&latitude=52.469913&longitude=-1.92736&signature=4lj2qj1xUhBRFJmzlY0iLfJc3cA=
52.469913
de=-1.92736&sign
{'PROPERTY TYPE': 'Semi-Detached'}
{'PROPERTY TYPE': 'Semi-Detached', 'Address': 'Harborne Road, Edgbaston', 'Price': '£1,699,950', 'Latitude': '52.469913', 'Longitude': 'de=-1.92736&sign', 'BEDROOMS': '×6'}
{'PROPERTY TYPE': 'Semi-Detached', 'Address': 'Harborne Road, Edgbaston', 'Price': '£1,699,950', 'Latitude': '52.469913', 'Longitude': 'de=-1.92736&sign', 'BEDROOMS': '×6', 'BATHROOMS': '×5'}
{'PROPERTY TYPE': 'Semi-Detached', 'Address': 'Harborne

---Currently on:https://www.rightmove.co.uk/properties/125464598#/?channel=RES_BUY----
---===Went back, Currently on:https://www.rightmove.co.uk/property-for-sale/find.html?searchType=SALE&locationIdentifier=REGION%5E162&insId=1&radius=0.0&minPrice=&maxPrice=&minBedrooms=&maxBedrooms=&displayPropertyType=&maxDaysSinceAdded=&_includeSSTC=on&sortByPriceDescending=&primaryDisplayPropertyType=&secondaryDisplayPropertyType=&oldDisplayPropertyType=&oldPrimaryDisplayPropertyType=&newHome=&auction=false===----
25
https://www.rightmove.co.uk/properties/126369464#/?channel=RES_BUY
url15:https://www.rightmove.co.uk/properties/126369464#/?channel=RES_BUY
fetched values
fetched keys
Hagley Road, Edgbaston, Birmingham, B17 8BJ
£1,500,000
<selenium.webdriver.remote.webelement.WebElement (session="1a61949b69cf07612d04cada3bb45c47", element="ca663f7f-83d6-4ab9-b797-09607359f03f")>

https://media.rightmove.co.uk/map/_generate?width=768&height=347&zoomLevel=15&latitude=52.472496&longitude=-1.966464&signa

{'PROPERTY TYPE': 'Detached', 'Address': 'Pebble Mill Road, Edgbaston', 'Price': '£1,300,000', 'Latitude': '52.452486', 'Longitude': 'de=-1.91311&sign', 'BEDROOMS': '×6', 'BATHROOMS': '×4', 'TENURE': 'Freehold'}
---Currently on:https://www.rightmove.co.uk/properties/121327259#/?channel=RES_BUY----
---===Went back, Currently on:https://www.rightmove.co.uk/property-for-sale/find.html?searchType=SALE&locationIdentifier=REGION%5E162&insId=1&radius=0.0&minPrice=&maxPrice=&minBedrooms=&maxBedrooms=&displayPropertyType=&maxDaysSinceAdded=&_includeSSTC=on&sortByPriceDescending=&primaryDisplayPropertyType=&secondaryDisplayPropertyType=&oldDisplayPropertyType=&oldPrimaryDisplayPropertyType=&newHome=&auction=false===----
25
https://www.rightmove.co.uk/properties/123552104#/?channel=RES_BUY
url20:https://www.rightmove.co.uk/properties/123552104#/?channel=RES_BUY
fetched values
fetched keys
Hagley Road West, Harborne, Birmingham, B17 8AL
£1,295,000
<selenium.webdriver.remote.webelement.WebElement (

---Currently on:https://www.rightmove.co.uk/properties/80177634#/?channel=RES_BUY----
---===Went back, Currently on:https://www.rightmove.co.uk/property-for-sale/find.html?searchType=SALE&locationIdentifier=REGION%5E162&insId=1&radius=0.0&minPrice=&maxPrice=&minBedrooms=&maxBedrooms=&displayPropertyType=&maxDaysSinceAdded=&_includeSSTC=on&sortByPriceDescending=&primaryDisplayPropertyType=&secondaryDisplayPropertyType=&oldDisplayPropertyType=&oldPrimaryDisplayPropertyType=&newHome=&auction=false===----
-------moving to next page https://www.rightmove.co.uk/property-for-sale/find.html?locationIdentifier=REGION%5E162&index=24&propertyTypes=&includeSSTC=false&mustHave=&dontShow=&furnishTypes=&keywords=--------
25
https://www.rightmove.co.uk/properties/126908858#/?channel=RES_BUY
url0:https://www.rightmove.co.uk/properties/126908858#/?channel=RES_BUY
fetched values
fetched keys
Washwood Heath Road, Birmingham
£325,000
<selenium.webdriver.remote.webelement.WebElement (session="1a61949b69cf07

{'PROPERTY TYPE': 'Detached', 'Address': 'Lordswood Road, Birmingham, B17', 'Price': '£1,125,000', 'Latitude': '52.471343', 'Longitude': 'de=-1.96848&sign', 'BEDROOMS': '×6', 'BATHROOMS': '×3', 'TENURE': 'Freehold'}
---Currently on:https://www.rightmove.co.uk/properties/125193164#/?channel=RES_BUY----
---===Went back, Currently on:https://www.rightmove.co.uk/property-for-sale/find.html?locationIdentifier=REGION%5E162&index=24&propertyTypes=&includeSSTC=false&mustHave=&dontShow=&furnishTypes=&keywords====----
25
https://www.rightmove.co.uk/properties/126748532#/?channel=RES_BUY
url5:https://www.rightmove.co.uk/properties/126748532#/?channel=RES_BUY
fetched values
fetched keys
Moorland Road, Edgbaston
£1,100,000
<selenium.webdriver.remote.webelement.WebElement (session="1a61949b69cf07612d04cada3bb45c47", element="8ff85363-e1da-4a7b-b8da-ba5ce3bcdfc3")>

https://media.rightmove.co.uk/map/_generate?width=768&height=347&zoomLevel=15&latitude=52.47452&longitude=-1.946496&signature=QH0t86Vukw

---===Went back, Currently on:https://www.rightmove.co.uk/property-for-sale/find.html?locationIdentifier=REGION%5E162&index=24&propertyTypes=&includeSSTC=false&mustHave=&dontShow=&furnishTypes=&keywords====----
25
https://www.rightmove.co.uk/properties/124923656#/?channel=RES_BUY
url10:https://www.rightmove.co.uk/properties/124923656#/?channel=RES_BUY
fetched values
fetched keys
Gilmorton Close, Harborne, Birmingham, B17 8QR
£1,095,000
<selenium.webdriver.remote.webelement.WebElement (session="1a61949b69cf07612d04cada3bb45c47", element="647987b2-c102-4eb9-9344-5260290e7c3b")>

https://media.rightmove.co.uk/map/_generate?width=768&height=347&zoomLevel=15&latitude=52.464108&longitude=-1.962187&signature=yIzGWAZoV_b0WyBY41mndY6lEj0=
https://media.rightmove.co.uk/map/_generate?width=768&height=347&zoomLevel=15&latitude=52.464108&longitude=-1.962187&signature=yIzGWAZoV_b0WyBY41mndY6lEj0=
52.464108
de=-1.962187&sig
{'PROPERTY TYPE': 'Detached'}
{'PROPERTY TYPE': 'Detached', 'Address': 'Gilmo

{'PROPERTY TYPE': 'Apartment', 'Address': 'Beaufort, Norfolk Road, Edgbaston, Birmingham, B15', 'Price': '£1,000,000', 'Latitude': '52.46779&', 'Longitude': 'e=-1.943654&sign', 'BEDROOMS': '×2', 'BATHROOMS': '×2', 'TENURE': 'Leasehold'}
---Currently on:https://www.rightmove.co.uk/properties/85059207#/?channel=RES_NEW----
---===Went back, Currently on:https://www.rightmove.co.uk/property-for-sale/find.html?locationIdentifier=REGION%5E162&index=24&propertyTypes=&includeSSTC=false&mustHave=&dontShow=&furnishTypes=&keywords====----
25
https://www.rightmove.co.uk/properties/126594893#/?channel=RES_BUY
url15:https://www.rightmove.co.uk/properties/126594893#/?channel=RES_BUY
fetched values
fetched keys
St Agnes Road, Moseley, Birmingham, B13
£1,000,000
<selenium.webdriver.remote.webelement.WebElement (session="1a61949b69cf07612d04cada3bb45c47", element="dbaf193e-90a0-491c-a1ac-91bfd287099a")>

https://media.rightmove.co.uk/map/_generate?width=768&height=347&zoomLevel=15&latitude=52.439031&lon

fetched values
fetched keys
Knightlow Road , Harborne
£995,000
<selenium.webdriver.remote.webelement.WebElement (session="1a61949b69cf07612d04cada3bb45c47", element="e2adda7f-c8b7-4e63-8510-3610d09d2bf6")>

https://media.rightmove.co.uk/map/_generate?width=768&height=347&zoomLevel=15&latitude=52.468422&longitude=-1.964107&signature=Zf8LGOiNFf5PCXYMwWoPVBqHsJo=
https://media.rightmove.co.uk/map/_generate?width=768&height=347&zoomLevel=15&latitude=52.468422&longitude=-1.964107&signature=Zf8LGOiNFf5PCXYMwWoPVBqHsJo=
52.468422
de=-1.964107&sig
{'PROPERTY TYPE': 'Detached'}
{'PROPERTY TYPE': 'Detached', 'Address': 'Knightlow Road , Harborne', 'Price': '£995,000', 'Latitude': '52.468422', 'Longitude': 'de=-1.964107&sig', 'BEDROOMS': '×4'}
{'PROPERTY TYPE': 'Detached', 'Address': 'Knightlow Road , Harborne', 'Price': '£995,000', 'Latitude': '52.468422', 'Longitude': 'de=-1.964107&sig', 'BEDROOMS': '×4', 'BATHROOMS': '×2'}
{'PROPERTY TYPE': 'Detached', 'Address': 'Knightlow Road , Harborne', '

In [11]:
df

,Address,BATHROOMS,BEDROOMS,Latitude,Longitude,PROPERTY TYPE,Price,SIZE,TENURE,SECTOR
0,"Sydenham Place, 26B Tenby Street , Jewellery Q...",×2,×3,52.48575&,e=-1.91351&signa,Penthouse,"£449,950","1,281 sq. ft.",Leasehold,NaN
1,"Suttons Drive, Great Barr, B43",NaN,NaN,52.559566,de=-1.927841&sig,Land,"£7,000,000",NaN,NaN,NaN
2,"Land at Mary Vale Road/Franklin Way, Bournvill...",NaN,NaN,52.42634&,e=-1.93486&signa,NaN,"£3,000,000",NaN,NaN,Land for sale
3,MIXED USE SITE WITH PLANNING FOR CONVERSION 47...,NaN,NaN,52.52244&,e=-1.84093&signa,NaN,"£2,699,999",NaN,Freehold,NaN
4,"Bearwood Parade, Bearwood High Street, West Mi...",×38,NaN,52.4731&l,=-1.96904&signat,Studio,"£2,400,000",NaN,Freehold,NaN
5,"Wellington Road, Edgbaston",×3,×6,52.462935,de=-1.909706&sig,Detached,"£2,250,000",NaN,Freehold,NaN
6,"Kings Close, Kings Norton, Birmingham, West Mi...",NaN,NaN,52.40808&,e=-1.92499&signa,House,"£2,000,000",NaN,Ask agent,NaN
7,"Bournbrook Road, Selly Oak, Birmingham, B29 7BU",×16,×16,52.44333&,e=-1.92757&signa,House,"£1,800,000",NaN,Freehold,NaN
8,"Handsworth Wood Road, Handsworth Wood, Birming...",×16,×19,52.516373,de=-1.925288&sig,NaN,"£1,700,000",NaN,Freehold,NaN
9,"Heeley Road, Birmingham",×8,×20,52.439765,de=-1.932093&sig,House,"£1,700,000",NaN,Freehold,NaN
